# Advent of Code 2021

Here, I will solve problems from Advent of Code 2021 and compare my solution with Norvig's. After a quick glance at his solutions, I make improvements on mine. 

Import libraries and utilities.

In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median
from math        import ceil, inf
from functools   import lru_cache
import matplotlib.pyplot as plt
import re

def answer(puzzle_number, got, expected) -> bool:
    """Verify the answer we got was the expected answer."""
    assert got == expected, f'For {puzzle_number}, expected {expected} but got {got}.'
    return True

# Now able to parse given strings
def parse(inp, parser=str, sep='\n', print_lines=7) -> tuple:
    """Split the day's input file into entries separated by `sep`, 
    and apply `parser` to each. If a string is given, parse it instad."""
    if isinstance(inp, int):
        title = f'input{inp}.txt' # Suited to my needs
        text  = open(title).read()
    elif isinstance(inp, str):
        title = 'sample'
        text = inp.strip()
    entries = mapt(parser, text.rstrip().split(sep))
    if print_lines:
        all_lines = text.splitlines()
        lines = all_lines[:print_lines]
        head = f'{title} ➜ {len(text)} chars, {len(all_lines)} lines; first {len(lines)} lines:'
        dash = "-" * 100
        print(f'{dash}\n{head}\n{dash}')
        for line in lines:
            print(trunc(line))
        print(f'{dash}\nparse({inp}) ➜ {len(entries)} entries:\n'
              f'{dash}\n{trunc(str(entries))}\n{dash}')
    return entries

def trunc(s: str, left=70, right=25, dots=' ... ') -> str: 
    """All of string s if it fits; else left and right ends of s with dots in the middle."""
#     dots = ' ... ' No need
    return s if len(s) <= left + right + len(dots) else s[:left] + dots + s[-right:]

Char = str # Intended as the type of a one-character string
Atom = Union[float, int, str]

def ints(text: str) -> Tuple[int]:
    """A tuple of all the integers in text, ignoring non-number characters."""
    return mapt(int, re.findall(r'-?[0-9]+', text))

def digits(text: str) -> Tuple[int]:
    """A tuple of all the digits in text (as ints 0–9), ignoring non-digit characters."""
    return mapt(int, re.findall(r'[0-9]', text))

def words(text: str) -> List[str]:
    """A list of all the alphabetic words in text, ignoring non-letters."""
    return re.findall(r'[a-zA-Z]+', text)

def atoms(text: str) -> Tuple[Atom]:
    """A tuple of all the atoms (numbers or symbol names) in text."""
    return mapt(atom, re.findall(r'[a-zA-Z_0-9.+-]+', text))

def atom(text: str) -> Atom:
    """Parse text into a single float or int or str."""
    try:
        x = float(text)
        return round(x) if round(x) == x else x
    except ValueError:
        return text
    
def mapt(fn, *args) -> tuple:
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

def quantify(iterable, pred=bool) -> int:
    """Count the number of items in iterable for which pred is true."""
    return sum(1 for item in iterable if pred(item))

class multimap(defaultdict):
    """A mapping of {key: [val1, val2, ...]}."""
    def __init__(self, pairs: Iterable[tuple], symmetric=False):
        """Given (key, val) pairs, return {key: [val, ...], ...}.
        If `symmetric` is True, treat (key, val) as (key, val) plus (val, key)."""
        self.default_factory = list
        for (key, val) in pairs:
            self[key].append(val)
            if symmetric:
                self[val].append(key)

def prod(numbers) -> float: # Will be math.prod in Python 3.8
    """The product formed by multiplying `numbers` together."""
    result = 1
    for x in numbers:
        result *= x
    return result

def total(counter: Counter) -> int: 
    """The sum of all the counts in a Counter."""
    return sum(counter.values())

def sign(x) -> int: return (0 if x == 0 else +1 if x > 0 else -1)

def transpose(matrix) -> list: return list(zip(*matrix))

def nothing(*args) -> None: return None

cat     = ''.join
flatten = chain.from_iterable
cache   = lru_cache(None)

Point = Tuple[int, int] # (x, y) points on a grid

neighbors4 = ((0, 1), (1, 0), (0, -1), (-1, 0))               
neighbors8 = ((1, 1), (1, -1), (-1, 1), (-1, -1)) + neighbors4

class Grid(dict):
    """A 2D grid, implemented as a mapping of {(x, y): cell_contents}."""
    def __init__(self, mapping=(), rows=(), neighbors=neighbors4):
        """Initialize with, e.g., either `mapping={(0, 0): 1, (1, 0): 2, ...}`,
        or `rows=[(1, 2, 3), (4, 5, 6)].
        `neighbors` is a collection of (dx, dy) deltas to neighboring points.`"""
        self.update(mapping if mapping else
                    {(x, y): val 
                     for y, row in enumerate(rows) 
                     for x, val in enumerate(row)})
        self.width  = max(x for x, y in self) + 1
        self.height = max(y for x, y in self) + 1
        self.deltas = neighbors
        
    def copy(self) -> Grid: return Grid(self, neighbors=self.deltas)
    
    def neighbors(self, point) -> List[Point]:
        """Points on the grid that neighbor `point`."""
        x, y = point
        return [(x+dx, y+dy) for (dx, dy) in self.deltas 
                if (x+dx, y+dy) in self]
    
    def to_rows(self) -> List[List[object]]:
        """The contents of the grid in a rectangular list of lists."""
        return [[self[x, y] for x in range(self.width)]
                for y in range(self.height)]

## Day 1

Problem 1.1: How many times the value increased from the previous one?

In [8]:
inp1 = parse(1, parser=int)

----------------------------------------------------------------------------------------------------
input1.txt ➜ 9777 chars, 2000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
151
152
153
158
159
163
164
----------------------------------------------------------------------------------------------------
parse(1) ➜ 2000 entries:
----------------------------------------------------------------------------------------------------
(151, 152, 153, 158, 159, 163, 164, 162, 161, 167, 169, 168, 169, 170, ... , 8078, 8081, 8112, 8127)
----------------------------------------------------------------------------------------------------


In [51]:
# A sample provided by Advent of Code
sample1 = [199, 200, 208, 210, 200, 207, 240, 269, 260, 263]

In [35]:
# My naive attempt to solve
def deeper(inputs):
    return sum(1 for x, y in zip(inputs[:-1], inputs[1:]) if y > x)

In [94]:
# Way better version with quantify adopted from Norvig
def deeper_quant(inputs):
    return quantify(inputs[i + 1] > inputs[i] for i in range(len(inputs) - 1))

In [92]:
deeper(sample1)

7

In [95]:
answer('sample1', deeper_quant(sample1), 7)

True

In [37]:
deeper(inp1)

1564

In [96]:
answer(1.1, deeper_quant(inp1), 1564)

True

Problem 1.2: Three-measurement sliding window. Instead of comparing one item to another, we now compare a tripplet to next tripplet.

In [54]:
# My naive version
def deeper2(inputs):
    three_measurements = [x+y+z for x, y, z in 
                          zip(inputs[:-2], inputs[1:-1], inputs[2:])]
    return sum(1 for x, y in zip(three_measurements[:-1], three_measurements[1:]) if y > x)

In [86]:
# Better version with quantify
def deeper_quant2(inputs):
    return quantify(sum(inputs[i:i+3]) < sum(inputs[i+1:i+4])
                   for i in range(len(inputs) - 3))

In [55]:
deeper2(sample1)

5

In [87]:
answer('sample1', deeper_quant2(sample1), 5)

True

In [56]:
deeper2(inp1[:10])

6

In [57]:
deeper2(inp1)

1611

In [88]:
answer('sample1', deeper_quant2(inp1), 1611)

True

## Day 2

Problem 2.1: What is the product of distance travelled? 

In [52]:
inp2 = parse(2, parser=lambda x: x.split() )

----------------------------------------------------------------------------------------------------
input2.txt ➜ 7793 chars, 1000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
forward 6
down 2
forward 2
down 8
forward 3
down 6
down 8
----------------------------------------------------------------------------------------------------
parse(2) ➜ 1000 entries:
----------------------------------------------------------------------------------------------------
(['forward', '6'], ['down', '2'], ['forward', '2'], ['down', '8'], ['f ... , '9'], ['forward', '8'])
----------------------------------------------------------------------------------------------------


In [53]:
point = [0, 0]
point

[0, 0]

In [55]:
def dive(point, direction):
    """Follow direction from the point."""
    heading, distance = direction
    distance = int(distance)
    if heading == 'forward':
        point[0] += distance
    elif heading == 'down':
        point[1] += distance
    elif heading == 'up':
        point[1] -= distance
    else:
        raise ValueError('Invalid direction')

In [103]:
sample_inp2 = '''
forward 5
down 5
forward 8
up 3
down 8
forward 2
'''
samples2 = parse(sample_inp2, parser=lambda x: x.split())

----------------------------------------------------------------------------------------------------
sample ➜ 48 chars, 6 lines; first 6 lines:
----------------------------------------------------------------------------------------------------
forward 5
down 5
forward 8
up 3
down 8
forward 2
----------------------------------------------------------------------------------------------------
parse(
forward 5
down 5
forward 8
up 3
down 8
forward 2
) ➜ 6 entries:
----------------------------------------------------------------------------------------------------
(('forward', 5), ('down', 5), ('forward', 8), ('up', 3), ('down', 8), ('forward', 2))
----------------------------------------------------------------------------------------------------


In [67]:
def solve2_1(directions):
    """Solve problem 2.1"""
    point = [0, 0]
    for direction in directions:
        dive(point, direction)
    return prod(point)

In [70]:
answer('sample 2.1', solve2_1(samples2), 150)

True

In [64]:
solve2_1(inp2)

2322630

Problem 2.2: Now, down increases aim by X, up decreases aim by X, and forward moves horizontal distance and increases depth by aim multiplied by X.

In [85]:
def dive2(point, direction):
    """Follow direction from the point with aim."""
    heading, distance = direction
    distance = int(distance)
    if heading == 'forward':
        point[0] += distance
        point[1] += point[2] * distance
    elif heading == 'down':
        point[2] += distance
    elif heading == 'up':
        point[2] -= distance
    else:
        raise ValueError('Invalid direction')

In [90]:
def solve2_2(directions):
    """Solve problem 2.2"""
    point = [0, 0, 0] # x, y, aim
    for direction in directions:
        dive2(point, direction)
    return point[0] * point[1]

In [92]:
answer('sample 2.2', solve2_2(samples2), 900)

True

In [93]:
solve2_2(inp2)

2105273490

I used a list to store x, y, and aim, but I did not have to. Because it is such a simple problem, Norvig did not bother making a list. When parsing the input, I splitted the inputs by using lambda function, but atoms function can do exactly what I wanted to achieve by parsing integer as int.

In [97]:
inp2 = parse(2, atoms)

----------------------------------------------------------------------------------------------------
input2.txt ➜ 7793 chars, 1000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
forward 6
down 2
forward 2
down 8
forward 3
down 6
down 8
----------------------------------------------------------------------------------------------------
parse(2) ➜ 1000 entries:
----------------------------------------------------------------------------------------------------
(('forward', 6), ('down', 2), ('forward', 2), ('down', 8), ('forward', ... own', 9), ('forward', 8))
----------------------------------------------------------------------------------------------------


In [104]:
sample_inp2 = '''
forward 5
down 5
forward 8
up 3
down 8
forward 2
'''
samples2 = parse(sample_inp2, atoms)

----------------------------------------------------------------------------------------------------
sample ➜ 48 chars, 6 lines; first 6 lines:
----------------------------------------------------------------------------------------------------
forward 5
down 5
forward 8
up 3
down 8
forward 2
----------------------------------------------------------------------------------------------------
parse(
forward 5
down 5
forward 8
up 3
down 8
forward 2
) ➜ 6 entries:
----------------------------------------------------------------------------------------------------
(('forward', 5), ('down', 5), ('forward', 8), ('up', 3), ('down', 8), ('forward', 2))
----------------------------------------------------------------------------------------------------


In [110]:
def improved_solve2_1(directions):
    x = y = 0
    for heading, n in directions:
        if heading == 'forward': x += n
        elif heading == 'down': y += n
        elif heading == 'up': y -= n
    return x * y

In [111]:
answer('improved sample 2.1', improved_solve2_1(samples2), 150)

True

In [112]:
answer('improved 2.1', improved_solve2_1(inp2), 2322630)

True

In [113]:
def improved_solve_2_2(directions):
    x = y = aim = 0
    for heading, n in directions:
        if heading == 'forward': 
            x += n
            y += n * aim
        elif heading == 'down': 
            aim += n
        elif heading == 'up': 
            aim -= n
    return x * y

In [114]:
answer('sample 2.2', solve2_2(samples2), 900)

True

In [115]:
answer('sample 2.2', solve2_2(inp2), 2105273490)

True

## Day 3

Problem 3.1: What is a value of the most and least common digits in binary, convert them to decimal, then multiplied together?

In [9]:
inp3 = parse(3, lambda x: x)

----------------------------------------------------------------------------------------------------
input3.txt ➜ 12999 chars, 1000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
100100110110
101110110110
010100010100
011001110000
000000000111
000010110001
001111000001
----------------------------------------------------------------------------------------------------
parse(3) ➜ 1000 entries:
----------------------------------------------------------------------------------------------------
('100100110110', '101110110110', '010100010100', '011001110000', '0000 ... 1110010', '111100110011')
----------------------------------------------------------------------------------------------------


In [20]:
nums = list(zip('1001', '0111', '0000'))
nums

[('1', '0', '0'), ('0', '1', '0'), ('0', '1', '0'), ('1', '1', '0')]

In [109]:
c = Counter(nums[0])
c.most_common()

[('0', 2), ('1', 1)]

In [39]:
c.most_common()[0][0]

'0'

In [112]:
list(c.values())

[1, 2]

In [28]:
int('0b10', 2)

2

In [35]:
sample_inp3 = '''
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
'''
samples3 = parse(sample_inp3)

----------------------------------------------------------------------------------------------------
sample ➜ 71 chars, 12 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
00100
11110
10110
10111
10101
01111
00111
----------------------------------------------------------------------------------------------------
parse(
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
) ➜ 12 entries:
----------------------------------------------------------------------------------------------------
('00100', '11110', '10110', '10111', '10101', '01111', '00111', '11100 ... 11001', '00010', '01010')
----------------------------------------------------------------------------------------------------


In [83]:
def get_commons(iterable, rank=0):
    """Grab the most common item from an iterable and convert it to decimal number."""
    return int(cat(Counter(inp).most_common()[rank][0] for inp in zip(*iterable)), 2)

In [88]:
def solve3_1(iterable):
    return get_commons(iterable) * get_commons(iterable, 1)

In [84]:
answer('sample3.1', get_commons(samples3), 22)

True

In [85]:
answer('sample3.1', get_commons(samples3, 1), 9)

True

In [89]:
answer('sample3.1', solve3_1(samples3), 198)

True

In [90]:
solve3_1(inp3)

2003336

Problem 3.2: We need to find O2 generator rating and CO2 scrubber rating. Here is how we find O2 generator rating. For first digit, find a most frequent digit, and filter numbers with that digit. Keep doing it until the last number is left or if there is a tie. If tie, keep the number with 1. To find CO2 scrubber rating, grab second frequent digits, and if there is a tie, grab a digit with 0. Then multiply O2 generator rating with CO2 scrubber rating.

In [172]:
l = list(range(10))
for n in l:
    if len(l) == 7:
        break
    if n % 2 == 0:
        l.remove(n)
    print(f'{n} checked')
l

0 checked
2 checked
4 checked


[1, 3, 5, 6, 7, 8, 9]

In [282]:
def get_commons2(iterable, rank=0):
    """For each digit, count occurrences and find a comparing number to compare.
    Compare the number with result and filter out ones that don't satisfy the condition."""
    result = list(iterable)
    for i in range(len(iterable[0])):
        if len(result) == 1:
            return result[0]
        compare = find_comparing_num(result, rank, i)
        result = list(filter(lambda x: x[i] == compare, result))
    return result[0]

def find_comparing_num(result, rank, i):
    """Find a number to compare with result by checking 'i'th digit of result."""
    most_common = Counter(res[i] for res in result).most_common()
    compare = most_common[rank][0] 
    if most_common[0][1] == most_common[1][1]:
        if rank == 0:
            compare = '1'
        else:
            compare = '0'
    return compare

In [283]:
answer(3.2, get_commons2(samples3), '10111')

True

In [284]:
answer(3.2, get_commons2(samples3, 1), '01010')

True

In [285]:
def solve3_2(inputs):
    return int(get_commons2(inputs), 2) * int(get_commons2(inputs, 1), 2)

In [286]:
answer('sample3.2', solve3_2(samples3), 230)

True

In [287]:
solve3_2(inp3)

1877139

I used Counter to count occurrences, but Norvig used str.count instead. Also, I used iterative approaches, but his version used recursive one. I love recursion, and I wish python transforms tail recursion into iterative version. I wrote my own recursive version just for fun.

In [290]:
def get_commons2_rec(binaries, rank=0, i=0):
    """Solve problem 3.2 recursively."""
    if len(binaries) == 1:
        return binaries[0]
    compare = find_comparing_num(binaries, rank, i)
    return get_commons2_rec(list(filter(lambda x: x[i] == compare, binaries)), rank, i + 1)

In [293]:
def solve3_2_rec(inputs):
    return int(get_commons2_rec(inputs), 2) * int(get_commons2_rec(inputs, 1), 2)

In [291]:
answer(3.2, get_commons2_rec(samples3), '10111')

True

In [292]:
answer(3.2, get_commons2_rec(samples3, 1), '01010')

True

In [294]:
answer('sample3.2', solve3_2_rec(samples3), 230)

True

In [295]:
answer(3.2, solve3_2_rec(inp3), 1877139)

True

## Day 4